<a href="https://colab.research.google.com/github/jaycrossler/ai-training/blob/main/Ollama%20and%20pydantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Outputs from pydantic-ai



## Step 1: Create a LLM server with ollama

To run this notebook, we need to have a OpenAI Compatible server. You can connect you own OpenAI account, huggingface CLI or use a local server. In the next cell, we will create an LLM server running on colab so that you dont' need to use any of the prior options.
> Note: If you are running this code on the Google Colab, be sure to check if you have a GPU (Runtime menu->`Change runtime type`->`gpu T4`).

In [1]:
# Download and install Ollama which will serve the LLM
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
# Importing nesseracy libraries
import subprocess
import time

In [3]:
# Start ollama in the background and use llama3.1 model

# Start the process in the background
server = subprocess.Popen(['ollama', 'serve'])
time.sleep(60) # To make sure ollama is ready in subsequent cell if you are running all not cell at a time

# To kill the server
# server.kill()

# To see all the models available: https://ollama.com/library
# Note: llama3.3 takes about 10 minutes to install and is slow to run (need more ram)
# Note: phi-4 and deepseek don't support tools in the pydantic format, so 3.2 seems best
MODEL = 'llama3.2'
llama3 = subprocess.Popen(['ollama', 'run', MODEL])
#time.sleep(90) # Make sure ollama is ready in subsequent cell if you are running all not cell at a time

# To kill the llama3
# llama3.kill()

In [4]:
subprocess.Popen(['ollama', 'run', 'phi4'])

<Popen: returncode: None args: ['ollama', 'run', 'phi4']>

In [6]:
# show which model(s) ollama is serving
!ollama list

NAME               ID              SIZE      MODIFIED       
llama3.2:latest    a80c4f17acd5    2.0 GB    33 seconds ago    


In [7]:
!pip install pydantic-ai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.5/252.5 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.5/262.5 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.2 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [8]:
from pydantic_ai import Agent

agent = Agent(
    'ollama:llama3.2'
    #system_prompt='Be concise, reply with one sentence.',
)

result = await agent.run('Where were the olympics held in 2012 ?')
print(result.data)

The 2012 Summer Olympics were held in London, United Kingdom.


In [9]:
from pydantic import BaseModel
from pydantic_ai import Agent

class CityLocation(BaseModel):
    city: str
    country: str

agent = Agent('ollama:llama3.2', result_type=CityLocation)

result = await agent.run('Which city and country were the olympics held in 2012?')
print(result.data)

city='London' country='United Kingdom'


In [10]:
from dataclasses import dataclass

from pydantic import BaseModel, Field

from pydantic_ai import Agent, RunContext, UnexpectedModelBehavior


class DatabaseConn:
    """This is a fake database for example purposes.

    In reality, you'd be connecting to an external database
    (e.g. PostgreSQL) to get information about customers.
    """

    @classmethod
    async def customer_name(cls, *, id: int) -> str | None:
        if id == 123:
            return 'John'

    @classmethod
    async def customer_balance(cls, *, id: int, include_pending: bool) -> float:
        if id == 123:
            return 123.45
        else:
            raise ValueError('Customer not found')


@dataclass
class SupportDependencies:
    customer_id: int
    db: DatabaseConn


class SupportResult(BaseModel):
    support_advice: str = Field(description='Advice returned to the customer')
    block_card: bool = Field(description='Whether to block their card')
    risk: int = Field(description='Risk level of query', ge=0, le=10)


support_agent = Agent(
    'ollama:llama3.2',
    deps_type=SupportDependencies,
    result_type=SupportResult,
    retries=20,
    system_prompt=(
        'You are a support agent in our bank, give the '
        'customer support and judge the risk level of their query. '
        "Reply using the customer's name."
    ),
)


@support_agent.system_prompt
async def add_customer_name(ctx: RunContext[SupportDependencies]) -> str:
    customer_name = await ctx.deps.db.customer_name(id=ctx.deps.customer_id)
    return f"The customer's name is {customer_name!r}"


@support_agent.tool
async def customer_balance(
    ctx: RunContext[SupportDependencies], include_pending: bool
) -> str:
    """Returns the customer's current account balance."""
    balance = await ctx.deps.db.customer_balance(
        id=ctx.deps.customer_id,
        include_pending=include_pending,
    )
    return f'${balance:.2f}'


if __name__ == '__main__':
    deps = SupportDependencies(customer_id=123, db=DatabaseConn())
    try:
      result = await support_agent.run('What is my name and balance?', deps=deps)
      print(result.data)
    except UnexpectedModelBehavior:
      print('Error')
      raise

    try:
      result = await support_agent.run('I just lost my card!', deps=deps)
      print(result.data)
    except UnexpectedModelBehavior:
      print('Error')
      raise


support_advice='Your current balance is $123.45.' block_card=False risk=1
support_advice='Please contact our 24/7 customer support team on 0800-123-4567 to report your lost card and reset your account information.' block_card=True risk=2


In [11]:
%pip install llama-index-llms-openai
%pip install llama-index-program-openai
%pip install llama-index-llms-llama-api
%pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.0/248.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 19.8 MB/s eta 0:00:00


In [12]:
from pydantic import BaseModel
from typing import List
from google.colab import userdata

from llama_index.program.openai import OpenAIPydanticProgram
from llama_index.llms.llama_api import LlamaAPI
import openai


openai.api_key = userdata.get('OPENAI_API_KEY')


In [13]:
class Song(BaseModel):
    """Data model for a song."""

    title: str
    length_seconds: int


class Album(BaseModel):
    """Data model for an album."""

    name: str
    artist: str
    songs: List[Song]

prompt_template_str = """\
Generate an example album, with an artist and a list of songs. \
Using the movie {movie_name} as inspiration.\
"""


program = OpenAIPydanticProgram.from_defaults(
    output_cls=Album, prompt_template_str=prompt_template_str, verbose=True
)
output = program(
    movie_name="Interstellar"
)

Function call: Album with args: {"name":"Interstellar Soundtrack","artist":"Hans Zimmer","songs":[{"title":"Dreaming of the Crash","length_seconds":221},{"title":"Cornfield Chase","length_seconds":284},{"title":"Dust","length_seconds":299},{"title":"Day One","length_seconds":221},{"title":"Stay","length_seconds":299},{"title":"Message from Home","length_seconds":221},{"title":"The Wormhole","length_seconds":284},{"title":"Mountains","length_seconds":299},{"title":"Afraid of Time","length_seconds":221},{"title":"A Place Among the Stars","length_seconds":299}]}


In [14]:
from pydantic import BaseModel
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key = userdata.get('OPENAI_API_KEY'))

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Jay is giving a presentation next Tuesday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

In [15]:
print(event)

name="Jay's Presentation" date='2023-10-24' participants=['Jay']
